In [ ]:
import pandas as pd
import numpy as np
import holoviews as hv
from holoviews import opts
import hvplot.pandas
import bokeh
import matplotlib.pyplot as plt
import matplotlib.animation as ani
import panel as pn
pn.extension()
hv.extension('bokeh')
from datetime import datetime, timezone
import geopy
import geopy.distance
import param

In [ ]:
eddy_min_longitude = -16
eddy_max_longitude = -13
eddy_min_latitude = 48
eddy_max_latitude = 50

observations = pd.read_csv('../data/observations.csv')
metadata = pd.read_csv('../data/metadata.csv')
metadata.sample_time = metadata.sample_time.apply(datetime.fromisoformat)
topic_probs = pd.read_csv('../data/topic_probs.csv', dtype=float)
word_probs = pd.read_csv('../data/word_probs.csv')
word_topic_matrix = pd.read_csv('../data/word_topic_matrix.csv')
oceanphysics = pd.read_csv('../data/oceanphysics.csv')
eddycenter = pd.read_csv('../data/eddycenter.csv')
oceanphysics.columns = ['sample_time', 'salinity', 'temperature', 'fluorescence', 'conductivity', 'sigma_t', 'instrument_date']
oceanphysics = oceanphysics.drop('instrument_date', axis=1)
oceanphysics.sample_time = oceanphysics.sample_time.apply(
    lambda x: datetime.strptime(x, '%d-%m-%Y %H:%M:%S').replace(tzinfo=timezone.utc)
)
eddycenter.sample_time = eddycenter.sample_time.apply(
    lambda x: datetime.strptime(x, '%d-%b-%Y %H:%M:%S').replace(tzinfo=timezone.utc)
)
metadata = metadata.set_index('sample_time')
oceanphysics = oceanphysics.set_index('sample_time').reindex(metadata.index, method='nearest')
eddycenter = eddycenter.set_index('sample_time').reindex(metadata.index, method='nearest')
topic_probs['sample_time'] = metadata.index
word_probs['sample_time'] = metadata.index
observations['sample_time'] = metadata.index
topic_probs = topic_probs.set_index('sample_time')
word_probs = word_probs.set_index('sample_time')
observations = observations.set_index('sample_time')
metadata_oceanphysics = pd.concat([metadata, oceanphysics], axis=1)
observations_norm = observations.div(observations.sum(axis=1), axis=-0)
topic_probs_idxmax = pd.DataFrame({
    'latitude': metadata.latitude,
    'longitude': metadata.longitude,
    'ml_topic': topic_probs.idxmax(axis=1)
})

In [ ]:
N = len(observations)

eddy_idx = metadata.longitude > eddy_min_longitude
eddy_idx &= metadata.longitude < eddy_max_longitude
eddy_idx &= metadata.latitude > eddy_min_latitude
eddy_idx &= metadata.latitude < eddy_max_latitude

timefmt = '%Y-%m-%d %H:%M:%S+%z'
epoch1_start = datetime.fromisoformat('2021-05-04 00:00:00+00:00')
epoch2_start = datetime.fromisoformat('2021-05-11 00:00:00+00:00')
epoch3_start = datetime.fromisoformat('2021-05-21 00:00:00+00:00')
epoch1_idx = metadata.index < epoch2_start
epoch2_idx = ~epoch1_idx & (metadata.index  < epoch3_start)
epoch3_idx = ~epoch1_idx & ~epoch2_idx

observations_epoch = pd.DataFrame(observations)
observations_epoch["epoch"] = (
    1 * (observations_epoch.index < epoch2_start) +
    2 * ((epoch2_start < observations_epoch.index) & (observations_epoch.index < epoch3_start)) +
    3 * (epoch3_start < observations_epoch.index)
)

In [ ]:
ship = [geopy.Point(*x) for x in zip(eddycenter.latitude, eddycenter.longitude)]
r_eddy = [geopy.distance.geodesic(kilometers=x) for x in eddycenter.r_ec]
b_eddy = [np.arctan2(y, x) for x, y in zip(eddycenter.x_ec, eddycenter.y_ec)]
eddy = [r.destination(s, b) for r, s, b in zip(r_eddy, ship, b_eddy)]
ship_eddy_df = pd.DataFrame(
    {
    "ship_longitude": eddycenter.longitude,
    "ship_latitude": eddycenter.latitude,
    "eddy_longitude": [x.longitude for x in eddy],
    "eddy_latitude": [x.latitude for x in eddy]
    },
    index=eddycenter.index
)

In [ ]:
class ShipEddyPlot:
    def __init__(
            self,
            df: pd.DataFrame,
            sx: str = 'ship_longitude',
            sy: str = 'ship_latitude',
            ex: str = 'eddy_longitude',
            ey: str = 'eddy_latitude',
            x_bounds: tuple[float, float] = (-25.0, -10.0),
            y_bounds: tuple[float, float] = (45.0, 55.0),
    ):
        self.df = df
        self.sx = sx
        self.sy = sy
        self.ex = ex
        self.ey = ey
        self.x_bounds = x_bounds
        self.y_bounds = y_bounds
        self.figure = None
        self.axis = None
        self.scatter = None

    def data(self, t):
        return (
            [self.df.loc[self.df.index[t], self.sx], self.df.loc[self.df.index[t], self.sy]],
            [self.df.loc[self.df.index[t], self.ex], self.df.loc[self.df.index[t], self.ey]]
        )

    def init(self, figure, axis):
        self.figure = figure
        self.axis = axis
        self.scatter = self.update(0)
        return self.scatter

    def update(self, t):
        ship, eddy = self.data(t)
        self.axis.clear()
        self.axis.scatter(
            *ship,
            marker='o',
            color='r',
        )
        self.scatter = self.axis.scatter(
            *eddy,
            marker='x',
            color='b',
        )
        self.axis.legend(['ship', 'eddy'])
        self.axis.set_xlim(self.x_bounds)
        self.axis.set_ylim(self.y_bounds)
        self.axis.grid(True)
        self.axis.set_title(f'{self.df.index[t]}')
        return self.scatter


In [ ]:
%matplotlib
shipeddyplot = ShipEddyPlot(ship_eddy_df)
fig, ax = plt.subplots()
a = ani.FuncAnimation(fig, shipeddyplot.update, frames=N,
                    init_func=lambda: shipeddyplot.init(fig, ax), blit=False)
plt.show()

